In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/long-practice/mlflow-basecode.git

Cloning into 'mlflow-basecode'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 218 (delta 107), reused 127 (delta 46), pack-reused 0
Receiving objects: 100% (218/218), 152.64 KiB | 1.59 MiB/s, done.
Resolving deltas: 100% (107/107), done.


In [3]:
%cd mlflow-basecode

/content/mlflow-basecode


In [4]:
import pandas as pd
import numpy as np

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [5]:
!pip install mlflow --quiet
!pip install optuna --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.2 MB/s eta 0:00:00


In [6]:
from src.pipeline import Pipeline
from preprocessing.preprocess import Preprocess

In [7]:
pipe = Pipeline(
    train_data=train,
    test_data=test,
    n_trial=10,
    test_size=0.2,
    use_mlflow=False
)
pipe.run()

INFO:preprocess:Preprocessing Set Missing Value
INFO:preprocess:Preprocessing Set Missing Value
INFO:preprocess:Preprocessing Remove Outlier
INFO:preprocess:column name is : id
INFO:preprocess:lower_bound is : -2703.0
INFO:preprocess:upper_bound is : 8109.0
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : CementComponent
INFO:preprocess:lower_bound is : -28.25000000000003
INFO:preprocess:upper_bound is : 616.95
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : BlastFurnaceSlag
INFO:preprocess:lower_bound is : -183.89999999999998
INFO:preprocess:upper_bound is : 306.5
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : FlyAshComponent
INFO:preprocess:lower_bound is : -118.5
INFO:preprocess:upper_bound is : 197.5
INFO:preprocess:----------------------------------------
INFO:preprocess:column name is : WaterComponent
INFO:preprocess:lower_bound is : 149.75
INFO:preprocess

In [8]:
import pickle

xgb_model = None
with open('./artifact/XGBoost.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

lgbm_model = None
with open('./artifact/LightGBM.pkl', 'rb') as f:
    lgbm_model = pickle.load(f)

submission = pd.read_csv('./data/sample_submission.csv')
# submission['Strength'] = xgb_model.predict(test)
submission['Strength'] = lgbm_model.predict(test)

In [9]:
submission

,id,Strength
0,5407,51.506403
1,5408,20.150636
2,5409,37.349403
3,5410,51.685313
4,5411,31.865182
...,...,...
3600,9007,32.848811
3601,9008,34.472438
3602,9009,43.603814
3603,9010,38.349417


In [10]:
import os

n = 0
submission_file_name_list = sorted([fname for fname in os.listdir('./prediction/') if fname.split('.')[-1] == 'csv'])
if submission_file_name_list:
    remove_tail = submission_file_name_list[-1].rstrip('.csv')
    n = int(remove_tail.split('_')[-1])
    n += 1

submission_path = './prediction/'
save_file_name = f'submission_{n}.csv'
file_path = os.path.join(submission_path, save_file_name)
submission.to_csv(file_path, index=False)